In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report
import re
import string
import warnings
warnings.filterwarnings('ignore')

In [2]:
fake=pd.read_csv("Fake.csv")
true=pd.read_csv("True.csv")

In [3]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [4]:
true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [5]:
fake['class']=0
true['class']=1

In [6]:
fake.shape,true.shape

((23481, 5), (21417, 5))

In [7]:
fake_manual=fake.tail(10)
for i in range(23480,23470,-1):
    fake.drop([i],axis=0,inplace=True)
true_manual=true.tail(10)
for i in range(21416,21406,-1):
    true .drop([i],axis=0,inplace=True)  

In [8]:
fake.shape,true.shape

((23471, 5), (21407, 5))

In [9]:
fake_manual['class']=0
true_manual['class']=1

In [10]:
merge=pd.concat([fake,true],axis=0)
merge.head()

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [11]:
merge.columns

Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')

In [12]:
data=merge.drop(['title','subject','date'],axis=1)

In [13]:
data.isnull().sum()

text     0
class    0
dtype: int64

In [14]:
data=data.sample(frac=1)
data.head()

,text,class
9013,WASHINGTON (Reuters) - Another attempt at gun ...,1
5116,Just when you thought Donald Trump s constant ...,0
16706,"LANY, Czech Republic (Reuters) - Czech billion...",1
19875,DUBAI (Reuters) - A gunfight erupted briefly i...,1
6106,"CALGARY, Alberta (Reuters) - A senior aide to ...",1


In [15]:
data.reset_index(inplace=True)
data.drop(['index'],axis=1,inplace=True)

In [16]:
data.columns

Index(['text', 'class'], dtype='object')

In [17]:
data.head()

,text,class
0,WASHINGTON (Reuters) - Another attempt at gun ...,1
1,Just when you thought Donald Trump s constant ...,0
2,"LANY, Czech Republic (Reuters) - Czech billion...",1
3,DUBAI (Reuters) - A gunfight erupted briefly i...,1
4,"CALGARY, Alberta (Reuters) - A senior aide to ...",1


In [18]:
def word(text):
    text=text.lower()
    text=re.sub('\[.*?\]','',text)
    text=re.sub('\\W'," ",text)
    text=re.sub('https?://\S+|www\.\S+'," ",text)
    text=re.sub('<.*?>+','',text)
    text=re.sub('[%s]' % re.escape(string.punctuation),'',text)
    text=re.sub('\n','',text)
    text=re.sub('\w*\d\w*','',text)
    return text

In [19]:
data['text']=data['text'].apply(word)

In [20]:
x=data['text']
y=data['class']

In [21]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25)

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [23]:
vector=TfidfVectorizer()
xy_train=vector.fit_transform(x_train)
xy_test=vector.transform(x_test)

In [24]:
from sklearn.linear_model import LogisticRegression

In [25]:
logReg=LogisticRegression()
logReg.fit(xy_train,y_train)

LogisticRegression()

In [26]:
pred=logReg.predict(xy_test)

In [27]:
logReg.score(xy_test,y_test)

0.985650623885918

In [28]:
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99      5955
           1       0.98      0.99      0.98      5265

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



In [29]:
from sklearn.tree import DecisionTreeClassifier

In [30]:
decT=DecisionTreeClassifier()

In [31]:
decT.fit(xy_train,y_train)
pred_DT=decT.predict(xy_test)
decT.score(xy_test,y_test)

0.9953654188948307

In [32]:
print(classification_report(y_test,pred_DT))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00      5955
           1       0.99      1.00      1.00      5265

    accuracy                           1.00     11220
   macro avg       1.00      1.00      1.00     11220
weighted avg       1.00      1.00      1.00     11220



In [33]:
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier

In [34]:
GB=GradientBoostingClassifier()
tree=RandomForestClassifier()

In [35]:
GB.fit(xy_train,y_train)

GradientBoostingClassifier()

In [36]:
pred_GB=GB.predict(xy_test)
GB.score(xy_test,y_test)

0.9944741532976827

In [37]:
print(classification_report(y_test,pred_GB))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99      5955
           1       0.99      1.00      0.99      5265

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



In [38]:
tree=RandomForestClassifier()
tree.fit(xy_train,y_train)

RandomForestClassifier()

In [39]:
pred_tree=tree.predict(xy_test)
tree.score(xy_test,y_test)

0.9874331550802139

In [40]:
print(classification_report(y_test,pred_tree))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5955
           1       0.98      0.99      0.99      5265

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



In [47]:
def output(n):
    if n==0:
        return "Fake news"
    else:
        return "True news"

def manual_testing(news):
    test_news={'text':[news]}
    new_def_test=pd.DataFrame(test_news)
    new_def_test['text']=new_def_test['text'].apply(word)
    new_x_test=new_def_test['text']
    new_xy_test=vector.transform(new_x_test)
    pred_LR=logReg.predict(new_xy_test)
    pred_DT=decT.predict(new_xy_test)
    pred_GB=GB.predict(new_xy_test)
    pred_tree=tree.predict(new_xy_test)
    return print('\n Logistic_Regression Prediction: {} \nDecision_Tree Prediction:{} \nGradient_Boosting Prediction:{} \nRandom_forest Prediction:{}'.format(output(pred_LR[0]),output(pred_DT[0]),output(pred_GB[0]),output(pred_tree[0])))

In [48]:
news=str(input())
manual_testing(news)

During the 2016 U.S. presidential election, the internet was abuzz with discussion when reports surfaced that Floyd Mayweather wore a hijab to a Donald Trump rally, daring people to fight him. The concocted story started on a sports comedy website, but it quickly spread on social media—and people took it seriously.

 Logistic_Regression Prediction: Fake news 
Decision_Tree Prediction:Fake news 
Gradient_Boosting Prediction:Fake news 
Random_forest Prediction:Fake news
